In [15]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm

https://tianchi.aliyun.com/competition/information.htm?spm=5176.100067.5678.2.4907341f5FWhlL&raceId=231656

In [16]:

test = pd.read_csv('round1_ijcai_18_test_b_20180418.txt', sep=' ')

train = pd.read_csv('round1_ijcai_18_train_20180301 (1).zip', sep=' ')

sample_sub = pd.read_csv('round1_ijcai_18_result_demo_20180301.txt', sep=' ')


In [32]:
# predict_category_property = train.predict_category_property.str.split(';',expand=True)
df = predict_category_property.copy()
for col in predict_category_property:
    df = pd.concat([df, predict_category_property[col].str.split(':',expand=True)], axis=1)
df.shape

(478138, 42)

In [3]:
import time
def preprocess(train):
    def timestamp2datetime(ts):
        dt = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ts))
        return dt
    train['Timestamp'] = train.context_timestamp.apply(timestamp2datetime)
    train = train.drop(['context_timestamp'], axis=1)
    def time_preprocess(X):
        X['Timestamp'] = pd.to_datetime(X['Timestamp'])
#         X['year'] = X['Timestamp'].dt.year
#         X['month'] = X['Timestamp'].dt.month 
        X['day'] = X['Timestamp'].dt.day
        X['week_day'] = X['Timestamp'].dt.weekday
        X['hour'] = X['Timestamp'].dt.hour
        X['minute'] = X['Timestamp'].dt.minute
        X['minute'] = X['minute'] // 15 * 15
        X = X.drop('Timestamp', axis=1)    
        return X
    train = time_preprocess(train)

    train['item_category_list'] = pd.to_numeric(train.item_category_list.str.split(';', expand=True)[1])

    df = train.item_property_list.str.split(';', expand=True)

    df = df.iloc[:, :21].fillna(-1).apply(pd.to_numeric)
    train = pd.concat([train, df], axis=1).reset_index(drop=True).drop('item_property_list', axis=1)

    train['predict_category_property'] = train.predict_category_property.str.split(';',
                                                                                   expand=True).isnull().sum(axis=1)
    train.columns = list(map(str, train.columns.tolist()))

    return train
train = preprocess(train)
test = preprocess(test)

In [4]:
from pandas_profiling import ProfileReport
profile = ProfileReport(train)
rejected_variables = profile.get_rejected_variables(threshold=0.9)
rejected_variables

/home/magas/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/pandas_profiling/plot.py:15: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/magas/.pyenv/versions/3.6.0/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/magas/.pyenv/versions/3.6.0/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/magas/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/magas/.pyenv/versions/3.6.0/envs/general/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File

['shop_score_delivery',
 'shop_score_description',
 'shop_star_level',
 'user_occupation_id',
 'user_star_level']

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    train.drop(['is_trade', 'instance_id'] + rejected_variables, axis=1).values, train['is_trade'].values,
    test_size=0.2)

from sklearn.metrics import log_loss
import lightgbm as lgb
import telepyth # push notif in telegram
%telepyth -t 1260389131217015787
%load_ext wurlitzer
verbose = 1
plot = 1
params = {
#     "max_bin": 1024,
#     "learning_rate": 0.01,
    "boosting_type": "dart",
    "objective": "binary",
    'num_iterations':1000,
    "metric": "binary",
    "num_leaves": 2**12,
    "verbose": 1,
    "min_data_in_leaf": 40,
#     "boost_from_average": True
    'early_stopping_round': 50,
    'bagging_fraction':0.7

}

d_train = lgb.Dataset(x_train, y_train)
d_valid = lgb.Dataset(x_test, label=y_test)
model = lgb.train(params, d_train, valid_sets=d_valid)

df_fi = pd.DataFrame(train.drop(['is_trade', 'instance_id'] + rejected_variables, axis=1).columns, columns=['feature'])
df_fi['importance'] = list(model.feature_importance('gain'))
df_fi.sort_values('importance', ascending=False, inplace=True)
# print(df_fi)
if plot:
    plt.figure()
    df_fi.head(10).plot(kind='barh', x='feature', y='importance')
#     plt.title('Roc_auc is {}'.format(roc_auc_score(y_test, y_pred)))
    plt.xlabel('relative importance')
    plt.show()

In [8]:
y_pred = model.predict(test.drop(rejected_variables, axis=1))

In [9]:
sub = pd.DataFrame(test.instance_id, columns=['instance_id'])
sub['predicted_score'] = y_pred
sub.to_csv('sub.txt', sep=' ', index=False)

https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters.rst

https://tianchi.aliyun.com/competition/uploadResult.htm?spm=5176.11165320.5678.6.250e3ce10SUPZj&raceId=231656